# Statoil ship or iceberg notebook

Working on data subset to get used to how to manipulate this data

Instructions from: https://www.kaggle.com/timolee/fastai-kaggle-starter-kit/notebook

# Attempting new learning framework
found at: https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-convnet-lb-0-1850/notebook


In [2]:
#Mandatory imports
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
#from sklearn.cross_validation import StratifiedKFold, StratifiedShuffleSplit, train_test_split
#from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import *
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras import layers
from keras.layers import *
from keras.models import Model
#from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense

from keras.datasets import cifar10
from keras import applications
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
#from keras.applications.xception import Xception
#from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, LSTM, Input
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [3]:
path = '/home/ubuntu/courses/deeplearning1/nbs/data/statoil/'

In [4]:
train = pd.read_json(path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(path+"test.json")

In [5]:
train.inc_angle = train.inc_angle.apply(lambda x:np.nan if x =='na' else x)

In [6]:
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [7]:
X_band_1 = np.array([np.array(band).astype('float32').reshape(75,75) for band in train.band_1])

In [8]:
X_band_2 = np.array([np.array(band).astype('float32').reshape(75,75) for band in train.band_2])
X_test_1 = np.array([np.array(band).astype('float32').reshape(75,75) for band in test.band_1])
X_test_2 = np.array([np.array(band).astype('float32').reshape(75,75) for band in test.band_2])

In [9]:
X_band_3 = (X_band_1+X_band_2)/2
X_test_3 = (X_test_1+X_test_2)/2



In [13]:
X_band_3.shape

(1604, 75, 75)

In [10]:
X_train = np.concatenate([X_band_1[:,:,:,np.newaxis],
                        X_band_2[:,:,:,np.newaxis],
                        X_band_3[:,:,:,np.newaxis]],axis=-1)

X_test = np.concatenate([X_test_1[:,:,:,np.newaxis],
                        X_test_2[:,:,:,np.newaxis],
                        X_test_3[:,:,:,np.newaxis]],axis=-1)

In [15]:
X_train.shape

(1604, 75, 75, 3)

## Import packages needed for processing

In [11]:
from keras.layers import Dense, LSTM, Input

In [12]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=64

In [13]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

In [22]:
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]
            
# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model
    

#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=4
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [23]:
preds=myAngleCV(X_train, X_angle, X_test)

('\n===================FOLD=', 0)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=sigmoid.0, inputs=[/input_5,...)`


Epoch 1/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0228 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/100
24/24 [==============================] - 26s 1s/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 8/100
20/24 [========================>.....] - ETA: 4s - loss: nan 

KeyboardInterrupt: 